In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
268,AFG,Asia,Afghanistan,2020-09-24,39170.0,25.0,42.571,1451.0,5.0,2.143,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
468,ALB,Europe,Albania,2020-09-24,12787.0,121.0,138.714,370.0,3.0,3.857,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
737,DZA,Africa,Algeria,2020-09-24,50400.0,186.0,204.857,1698.0,9.0,7.571,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-23,France,473078.0,0.0,31473.0,0.0,0.0,0.0
0,2020-09-23,China,90418.0,0.0,4742.0,0.0,0.0,0.0
0,2020-09-23,Italy,302112.0,0.0,35771.0,0.0,0.0,0.0
0,2020-09-23,Spain,697257.0,0.0,31058.0,0.0,0.0,0.0
0,2020-09-23,United States,6972512.0,0.0,201699.0,0.0,0.0,0.0
0,2020-09-23,World,32215917.0,0.0,980322.0,0.0,0.0,0.0
0,2020-09-23,United Kingdom,406755.0,0.0,41864.0,0.0,0.0,0.0
0,2020-09-23,Germany,277524.0,0.0,9422.0,0.0,0.0,0.0
0,2020-09-23,Iran,431846.0,0.0,24867.0,0.0,0.0,0.0
0,2020-09-23,Turkey,307637.0,0.0,7694.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-24,France,473078.0,481141.0,31473.0,31459.0,-8063.0,14.0
0,2020-09-24,China,90418.0,90409.0,4742.0,4738.0,9.0,4.0
0,2020-09-24,Italy,302112.0,302537.0,35771.0,35758.0,-425.0,13.0
0,2020-09-23,Spain,697257.0,693556.0,31058.0,31034.0,3701.0,24.0
0,2020-09-24,United States,6972512.0,6934204.0,201699.0,201909.0,38308.0,-210.0
0,2020-09-24,World,32215917.0,31969439.0,980322.0,978284.0,246478.0,2038.0
0,2020-09-24,United Kingdom,406755.0,409729.0,41864.0,41862.0,-2974.0,2.0
0,2020-09-24,Germany,277524.0,278070.0,9422.0,9428.0,-546.0,-6.0
0,2020-09-24,Iran,431846.0,432798.0,24867.0,24840.0,-952.0,27.0
0,2020-09-24,Turkey,307637.0,308069.0,7694.0,7711.0,-432.0,-17.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")